In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# One-hot encode labels
y_train_cat = to_categorical(y_train, 10)
y_test_cat  = to_categorical(y_test, 10)

# Build TensorFlow datasets
batch_size = 64

def preprocess(image, label):
    image = tf.image.resize(image, (96, 96))       # resize here
    image = preprocess_input(image)                # mobilenet preprocess
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train_cat))
train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test_cat))
test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load base MobileNetV2 (pretrained on ImageNet, exclude top)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(96,96,3))
base_model.trainable = False  # Freeze base for transfer learning

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)  # regularization
output = Dense(10, activation="softmax")(x)  # CIFAR-10 has 10 classes
model = Model(inputs=base_model.input, outputs=output)

# Compile
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Train first phase (frozen base)
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 50s 40ms/step - accuracy: 0.3589 - loss: 2.0515 - val_accuracy: 0.7751 - val_loss: 0.6717
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 17ms/step - accuracy: 0.7371 - loss: 0.7850 - val_accuracy: 0.8140 - val_loss: 0.5345
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - accuracy: 0.7854 - loss: 0.6319 - val_accuracy: 0.8315 - val_loss: 0.4871
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.8063 - loss: 0.5709 - val_accuracy: 0.8406 - val_loss: 0.4592
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.8193 - loss: 0.5311 - val_accuracy: 0.8477 - val_loss: 0.4427
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.8255 - loss: 0.5049 - val_accuracy: 0.8493 - val_loss: 0.4331
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.8350 - loss: 0.4819 - val_accuracy: 0.8528 - val_l

In [2]:
model.save("mobilenetv2_colab_v1.keras")

In [4]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# One-hot encode labels (since we'll use categorical_crossentropy)
y_train_cat = to_categorical(y_train, 10)
y_test_cat  = to_categorical(y_test, 10)

# ===== Settings =====
batch_size = 64
IMG_SIZE = 224   # EfficientNetB0 default. If Colab OOMs, try 128 or 96.

# tf.data preprocessing for EfficientNetB0
# (Modern Keras EfficientNet includes Rescaling(1./255) internally,
#  so feed 0..255 floats; no external preprocess needed.)
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

train_ds = (tf.data.Dataset.from_tensor_slices((X_train, y_train_cat))
            .shuffle(10000)
            .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE))

test_ds = (tf.data.Dataset.from_tensor_slices((X_test, y_test_cat))
           .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
           .batch(batch_size)
           .prefetch(tf.data.AUTOTUNE))

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Base: EfficientNetB0 (pretrained, no top)
base_model = EfficientNetB0(weights="imagenet",
                            include_top=False,
                            input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # freeze for transfer learning

# Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)           # regularization
output = Dense(10, activation="softmax")(x)  # CIFAR-10 has 10 classes
model = Model(inputs=base_model.input, outputs=output)

# Compile
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",  # matches one-hot labels
    metrics=["accuracy"]
)

# Train first phase (frozen base)
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)

# Optional: fine-tune the top of the base with a lower LR
base_model.trainable = True
for layer in base_model.layers[:-30]:   # unfreeze last ~30 layers
    layer.trainable = False

model.compile(optimizer=Adam(1e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

history_ft = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 111s 107ms/step - accuracy: 0.4527 - loss: 1.6956 - val_accuracy: 0.8253 - val_loss: 0.6779
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 97s 72ms/step - accuracy: 0.8059 - loss: 0.6803 - val_accuracy: 0.8527 - val_loss: 0.5026
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 83s 74ms/step - accuracy: 0.8328 - loss: 0.5383 - val_accuracy: 0.8658 - val_loss: 0.4404
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 58s 74ms/step - accuracy: 0.8458 - loss: 0.4805 - val_accuracy: 0.8731 - val_loss: 0.4075
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 72ms/step - accuracy: 0.8555 - loss: 0.4467 - val_accuracy: 0.8782 - val_loss: 0.3854
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 72ms/step - accuracy: 0.8587 - loss: 0.4270 - val_accuracy: 0.8812 - val_loss: 0.3703
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 56s 72ms/step - accuracy: 0.8644 - loss: 0.4059 - val_accuracy: 0.8825 - val_loss: 0.3590
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 72ms/step - accuracy: 0.8687 - loss: 0.3921 

In [5]:
model.save("efficientnetb0_colab_v1.keras")

In [7]:
loss, acc = model.evaluate( test_ds, verbose=0)
print(f"Test accuracy: {acc:.4f}")

Test accuracy: 0.9267
